# Ochrana přístupu k vlastnostem objektu
Jsou případy, kdy chceme některé vlastnosti objektu ochránit před změnou nebo špatnou manipulací. Příkladem může být objem nádrže automobilu, který nemůže být menší než nula nebo při tankování doplnit více než je maximum, ale nám nic nebrání v tom to nastavit.

## Pomocí konvence
Některé jazyky (Java, TypeScript, C#, ...) mají modifikátory přístupu ([Access modifiers](https://en.wikipedia.org/wiki/Access_modifiers)), které nám umožňují schovat atribut objektu před vnějším použitím, Python to bohužel neumí.

Proto **existuje konvence použití podtržítka \"_\" před názvem atributu**.
Tato konvence říká, že atribut je **interní, ale nic nebrání jejímu použití**. 

In [1]:
class Auto:
    
    def __init__(self, znacka: str, objem_nadrze: int):
        self.znacka = znacka
        
        assert objem_nadrze > 0, "Objem nádrže nesmí být menší než 0"
        self._objem_nadrze = objem_nadrze
    
    def print(self) -> None:
        print(f"Auto značky {self.znacka} s objemem nádrže {self._objem_nadrze} litrů")

#auto = Auto("Volvo", -5); # Vyhodí chybu AssertionError: Objem nádrže nesmí být menší než 0
auto = Auto("Volvo", 70);
auto.print()

auto._objem_nadrze = -10
auto.print()

Auto značky Volvo s objemem nádrže 70 litrů
Auto značky Volvo s objemem nádrže -10 litrů


- Také **existuje konvence použití dvou podtržítek \"__\" před názvem atributu**.
- Tato konvence říká, že atribut je **chráněný, ale i tak se dá použít**.
- Interpret nám nezpřístupní daný atribut, ale dojde k jeho přejmenování na **\"_Auto__objemNadrze\"**.
- Je možné si dohledat atributy instance zobrazením slovníku auto.**\_\_dict\_\_**

In [2]:
class Auto:
    
    def __init__(self, znacka: str, objem_nadrze: int):
        self.znacka = znacka
        
        assert objem_nadrze > 0, "Objem nádrže nesmí být menší než 0"
        self.__objem_nadrze = objem_nadrze
    
    def print(self) -> None:
        print(f"Auto značky {self.znacka} s objemem nádrže {self.__objem_nadrze} litrů")

auto = Auto("Volvo", 70);
print(auto.__dict__)
auto.print()

auto.__objem_nadrze = -10
auto.print()

auto._Auto__objem_nadrze = -10
auto.print()

{'znacka': 'Volvo', '_Auto__objem_nadrze': 70}
Auto značky Volvo s objemem nádrže 70 litrů
Auto značky Volvo s objemem nádrže 70 litrů
Auto značky Volvo s objemem nádrže -10 litrů


## Pomocí metod
V Jazycích, kde fungují modifikátory přístupu se často používájí get a set metody.
A veškerá manipulace atributů probíhá skrze tyto metody. 

In [4]:
class Auto:
    
    def __init__(self, znacka: str, objemNadrze: int):
        self.znacka = znacka
        self.set_objem_nadrze(objemNadrze)
        
    def set_objem_nadrze(self, objem_nadrze: int):
        assert objem_nadrze > 0, "Objem nádrže nesmí být menší než 0"
        self._objem_nadrze = objem_nadrze
        
    def get_objem_nadrze(self) -> int:
        return self._objem_nadrze
    
    def print(self) -> None:
        print(f"Auto značky {self.znacka} s objemem nádrže {self._objem_nadrze} litrů")

auto = Auto("Volvo", 70);
auto.print()

auto.set_objem_nadrze(55)
print(f"Objem nádrže je {auto.get_objem_nadrze()} litrů")

auto._objem_nadrze = -10
auto.print()

Auto značky Volvo s objemem nádrže 70 litrů
Objem nádrže je 55 litrů
Auto značky Volvo s objemem nádrže -10 litrů


**Otázky**

Pokud bychom get/set metody přidávali do existujícího kódu. Bylo by dobré přepsat všechna použití a to by mohlo být náročné. Abychom to nemuseli dělat, můžeme použít funkci [**property**](). Tato funkce zajistí, že při jakémkoliv přístupu k atributu, dojde k zavolání get nebo set metody.

**⚠️property musí mít jiný název než atribut jinak dojde k zacyklení!⚠️**

In [6]:
class Auto:
    
    def __init__(self, znacka: str, objem_nadrze: int):
        self.znacka = znacka
        self.objem_nadrze = objem_nadrze
        
    def set_objem_nadrze(self, objem_nadrze: int):
        assert objem_nadrze > 0, "Objem nádrže nesmí být menší než 0"
        self._objem_nadrze = objem_nadrze
        
    def get_objem_nadrze(self) -> int:
        return self._objem_nadrze
    
    def print(self) -> None:
        print(f"Auto značky {self.znacka} s objemem nádrže {self._objem_nadrze} litrů")
    
    objem_nadrze = property(get_objem_nadrze, set_objem_nadrze)

auto = Auto("Volvo", 70);
auto.print()

auto.objem_nadrze = 55
print(f"Objem nádrže je {auto.objem_nadrze} litrů")

#auto.objem_nadrze = -10 # toto vyhodí chybu
#auto.print()

Auto značky Volvo s objemem nádrže 70 litrů
Objem nádrže je 55 litrů


**Otázky**

## Pomocí metod s dekorátorem
[Dekorátor](https://docs.python.org/3/glossary.html#term-decorator) je funkce vracející funkci a umožnuje nám přidat logiku k existující metodě.


**⚠️getter decorátor musí být deklarován před setter decorátorem!⚠️**\
Jinak dostanete chybu **NameError: name 'objem_nadrze' is not defined**

In [7]:
class Auto:
    
    def __init__(self, znacka: str, objem_nadrze: int):
        self.znacka = znacka
        self.objem_nadrze = objem_nadrze
    
    @property
    def objem_nadrze(self) -> int:
        return self._objem_nadrze
    
    @objem_nadrze.setter
    def objem_nadrze(self, objem_nadrze: int):
        assert objem_nadrze > 0, "Objem nádrže nesmí být menší než 0"
        self._objem_nadrze = objem_nadrze
    
    def print(self) -> None:
        print(f"Auto značky {self.znacka} s objemem nádrže {self._objem_nadrze} litrů")

auto = Auto("Volvo", 70);
auto.print()

auto.objem_nadrze = 55
print(f"Objem nádrže je {auto.objem_nadrze} litrů")

Auto značky Volvo s objemem nádrže 70 litrů
Objem nádrže je 55 litrů


**Otázky**

### Co používat
- Pokud máte atributy, které chcete chránit vždy používejte konvence
- U chráněných atributů používejte kotrolu pomocí metod.
    - Použití dekorátorů je rozšířenější, jednoduché a přehledné
    
**Otázky**

# Typy metod v Python OOP
**Otázky**
* Co si představíte pod typy method v OOP?
* Kolik typů máme?

Dejme tomu, že máme zákazníka pronajímajícího auta, který si vede evidenci v excelu.\
Vzhledem k tomu, že se mu daří auta, zakázky přibývají tak to začíná být neúnosné.\
Proto se rozhodl to převést do systému/programu.

**Úkol**\
Naprogramujte načítání záznamů excel souboru ve formátu csv `<znacka>,<objem_nadrze>`

In [8]:
auta_csv = """Volvo,70
            Nissan,45
            Audi,65"""

radky = auta_csv.split()
auta = []
for radek in radky:
    znacka, objem_nadrze = radek.split(",")
    auto = Auto(znacka, int(objem_nadrze))
    auta.append(auto)
    
for auto in auta:
    auto.print()

Auto značky Volvo s objemem nádrže 70 litrů
Auto značky Nissan s objemem nádrže 45 litrů
Auto značky Audi s objemem nádrže 65 litrů


Uvedený kód je krásná ukázka procedurálního programování.

Pokud bychom chtěli přepsat kód na objektovější, můžeme udělat dvě věci:
- Přesunout vytváření objektu auto do třídy Auto. Metodě, která vytváří instanci třídy se říka [Factory method](https://en.wikipedia.org/wiki/Factory_method_pattern)
- Vytvořit novou třídu CsvMapovac a přesunout do ní ostatní logiku načítání 

In [9]:
class Auto:
    
    def __init__(self, znacka: str, objem_nadrze: int):
        self.znacka = znacka
        self.objem_nadrze = objem_nadrze
    
    @property
    def objem_nadrze(self) -> int:
        return self._objem_nadrze
    
    @objem_nadrze.setter
    def objem_nadrze(self, objem_nadrze: int):
        assert objem_nadrze > 0, "Objem nádrže nesmí být menší než 0"
        self._objem_nadrze = objem_nadrze
    
    @classmethod
    def nacti_z_csv(cls, radek: str): # metoda třídy # Factory method
        znacka, objem_nadrze = radek.split(",")
        return cls(znacka, int(objem_nadrze))
    
    def print(self) -> None: # metoda instance
        print(f"Auto značky {self.znacka} s objemem nádrže {self._objem_nadrze} litrů")

class CsvMapovac:
    
    @staticmethod
    def mapuj(auta_csv: str, cls): # statická metoda
        auta = []
        for radek in radky:
            auto = cls.nacti_z_csv(radek)
            auta.append(auto)
        return auta
    
auta_csv = """Volvo,70
Nissan,45
Audi,65"""

auta = CsvMapovac.mapuj(auta_csv, Auto)
for auto in auta:
    auto.print()

Auto značky Volvo s objemem nádrže 70 litrů
Auto značky Nissan s objemem nádrže 45 litrů
Auto značky Audi s objemem nádrže 65 litrů


**Otázky**

V předchozím kódu si můžete všimnout použití ```@classmethod``` a ```@staticmethod```.\
V Pyhon OOP rozeznáváme tři typy metod:
- Instanční
    - Má parametr ```self``` odkazující na konkrétní **object**
    - Neznačí se dekorátorem
    - Má přístup k instanci i třídě
- Třídní 
    - Má parametr ```cls``` odkazující na **třídu**
    - Značí se dekorátorem ```@classmethod```
    - Má přístup pouze k třídě
- Statická
    - Nemá ```self``` ani ```cls``` parametr
    - Značí se dekorátorem ```@staticmethod```
    - Nemá přísutp k třídě ani instanci
    
## Co používat
Osobně bych volil jednoduché pravidlo:
- Zvolim instanční metodu
- Nepoužívám parametr ```self``` udělám z ní třídní metodu
- Nepoužívám parametr ```cls``` udělám z ní statickou metodu

**Otázky**
**Úkol**
* Rozšířit obě varianty o další atributy a validace
* Přidat načítání dodávek s nosnostní a objemem

# Shrnutí
* Python nemá modifikátory přístupu
* Použítím podtržítka před názvem říkáme atribut je interní a něl bys ho použít
* Použítí dvou podtržítek říká atribut je chráněný a kompilátor změní jeho název
* Přístup k atributům můžeme upravit i pomocí method get a set
* V dnešní době je rozšířenější řešení pomocí dekorátorů
    * Pracujeme jako s atributem, ale volají se metody
* Máme tři typy method v Python OOP
    * Instanční - má parametr `self` odkazující na volaný objekt
    * Třídní - má parametr `cls` odkazující na třídu, značí se `@classmethod`
    * Statická - nemá `self` a `cls` parametr, značí se `@staticmethod`